<img src="./intro_images/MIE.PNG" width="100%" align="left" />

<table style="float:right;">
    <tr>
        <td>                      
            <div style="text-align: right"><a href="https://alandavies.netlify.com" target="_blank">Dr Alan Davies</a></div>
            <div style="text-align: right">Senior Lecturer health data science</div>
            <div style="text-align: right">University of Manchester</div>
         </td>
         <td>
             <img src="./intro_images/alan.PNG" width="30%" />
         </td>
     </tr>
</table>

# 6.0 Using SQL in Python
****

#### About this Notebook
This notebook introduces the concept of using SQL in another programming language (Python). This includes using the SQL directly or via Object Relational Mapping (ORM).

<div class="alert alert-block alert-warning"><b>Learning Objectives:</b> 
<br/> At the end of this notebook you will be able to:
    
- Investigate key features of applying SQL in Python

- Explore and practice using ORM in Python

</div> 

<a id="top"></a>

<b>Table of contents</b><br>

6.1 [Using SQL in Python](#sql)

6.2 [Object-Relational Mapping](#orm)

We have looked at Python in previous notebooks and in this series at using SQL. Here we look at how we can use both Python and SQL together. This is typical of many modern programs that use databases at their core. Instead of using the SQL inline in the notebook we will now use the Python <code>sqlite3</code> library to connect to a database file.

<div class="alert alert-success">
<b>Note:</b> The previous knowledge gained about how SQL works will help make more sense of how to use SQL in Python. In some cases it may be easier to first write the code in SQL and transfer it into Python until you become more familiar with the Python libraries.
</div>

<a id="sql"></a>
#### 6.1 Using SQL in Python

In [1]:
import sqlite3
med_data_db = sqlite3.connect("medical_db.db") 
cursor = med_data_db.cursor()
cursor.execute("SELECT * FROM med_data;")
results = cursor.fetchall()

for result in results:
    print(result)
    
cursor.close()
med_data_db.close()

(1, 'Alan Smith', 24, 'M', '120/70', 78)
(2, 'Maureen Gdiver', 87, 'F', '156/82', 82)
(3, 'Adam Blythe', 54, 'M', '132/73', 72)
(4, 'Darren Sanders', 34, 'M', '155/67', 67)
(5, 'Sally-Ann Joyce', 19, 'F', '121/72', 65)


There is a lot going on here. Firstly I made a database file called <code>medical_db.db</code> and I recreated the <code>med_data</code> table we created in previous examples in that file. Next we load the <code>sqlite3</code> library. We then make a connection to the database using a variable I am calling <code>med_data_db</code>. We then make a <code>cursor</code>. This essentially lets us iterate over a set of records. We then use this to execute an SQL command. The command <code>SELECT * FROM med_data;</code> should be familiar to you by now. We then store the results in a variable and iterate through them in a loop. Finally we close the cursor and the connection to the database when we are done.

Now we have the data stored in the <code>results</code> variable, we can access it like a regular Python data structure. So I can view all of the record or a specific part:

In [4]:
print(results[0])

(1, 'Alan Smith', 24, 'M', '120/70', 78)


In [5]:
print(results[0][1])

Alan Smith


In [7]:
print("Patient:", results[0][1], "heart rate =" ,results[0][5])

Patient: Alan Smith heart rate = 78


<div class="alert alert-danger">
<b>Note:</b> We wouldn't typically try to read a whole database into a single variable. This might not be possible with a very large database and also would be very inefficient. Instead we would perform a query to return a subset of the data we are interested in using.
</div>

This lets us integrate data stored in a database with the programs we create. Hopefully you can see how useful such methods are for building a complete application that interacts with data, which is a core component of software systems and used frequently in both data science and informatics projects. 

<div class="alert alert-block alert-info">
<b>Task 1:</b>
<br> 
Print out the blood pressure for patient <code>Darren Sanders</code> with the patients name and unit of measure (mmHG).
</div>

In [8]:
print(results[3][1], "blood pressure =", results[3][4], "mmHG")

Darren Sanders blood pressure = 155/67 mmHG


In [ ]:
# Type your code here


<div class="alert alert-block alert-info">
<b>Task 2:</b>
<br> 
Write a query to return all the patients details for patients over the age of 50 years. Print out the returned values with the Python <code>print()</code> function.
</div>

In [16]:
med_data_db = sqlite3.connect("medical_db.db") 
cursor = med_data_db.cursor()
cursor.execute("SELECT * FROM med_data WHERE Age > 50;")
results = cursor.fetchall()

for result in results:
    print(result)
    
cursor.close()
med_data_db.close()

(2, 'Maureen Gdiver', 87, 'F', '156/82', 82)
(3, 'Adam Blythe', 54, 'M', '132/73', 72)


In [ ]:
# Type your code here


<a id="orm"></a>
#### 6.2 Object-Relational Mapping

The only issue with this approach is that you tend to have a mixture of Python and SQL code intermixed in your code. This also requires you to be proficient in both SQL and Python and can also be harder to maintain. One way to overcome this is to use a technique called <code>Object-Relational Mapping (ORM)</code>. This works by converting the data  using Object Orientated Programming techniques to make the conversion between otherwise incompatible systems. The mapping process maps tables in an SQL database to objects in Python. 

The figure below shows how this works. From the programmers perspective, they only ever deal with writing to and from Python objects (or whatever language they are using). The Object mapper converts this to SQL and queries the database returning data and converting it into objects. This way a developer doesn't need to mix SQL and Python in the same program. This also makes it easier to swap out database engines (say you wanted to change form SQLite to MySQL for instance) without rewriting large amounts of code.

<img src="./intro_images/orm.PNG" width="100%" />

Lets look at an example. First we will make the object to store the data in. We make one object per table in the database.

In [2]:
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, select
engine = create_engine('sqlite:///medical_db.db', echo = False)
meta = MetaData()

med_data = Table(
   'med_data', meta, 
   Column('id', Integer, primary_key = True), 
   Column('Name', String), 
   Column('Age', Integer),
   Column('Sex', String),
   Column('Blood pressure', String), 
   Column('Heart rate', String),
)
meta.create_all(engine)

Here we import the <code>sqlalchemy</code> library and then load the database. We use the <code>Table</code> and <code>Column</code> functions to create an object to store the data from the SQL table along with its associated data type. Note that we are using an <code>SQLite</code> engine. We could swap this for a different vendor i.e. PostgreSQL or MySQL.

<div class="alert alert-success">
<b>Note:</b> By setting <code>echo = False</code> in the <code>create_engine</code> function, we just see the output. If we set it to <code>True</code>, we would also see the SQL statements it is executing behind the scenes. If you want to see what that looks like, change to <code>echo = True</code> and rerun the cell.
</div>

In [3]:
data = med_data.select()
conn = engine.connect()
result = conn.execute(data)

for each_row in result:
    print(each_row)

(1, 'Alan Smith', 24, 'M', '120/70', 78)
(2, 'Maureen Gdiver', 87, 'F', '156/82', 82)
(3, 'Adam Blythe', 54, 'M', '132/73', 72)
(4, 'Darren Sanders', 34, 'M', '155/67', 67)
(5, 'Sally-Ann Joyce', 19, 'F', '121/72', 65)


we can then create a command and execute it. We store the result in a variable that we can iterate through as before. Now we can perform queries just using Python code. This keeps all our code in a project in the same language. Below we see an example query to find all the females in the table (note the use of the Python double equal operator for equality). 

In [4]:
females = med_data.select().where(med_data.c.Sex == 'F')
result = conn.execute(females)

for each_row in result:
    print(each_row)

(2, 'Maureen Gdiver', 87, 'F', '156/82', 82)
(5, 'Sally-Ann Joyce', 19, 'F', '121/72', 65)


This query returns the 2 records where the <code>Sex</code> is female.  

<div class="alert alert-block alert-info">
<b>Task 3:</b>
<br> 
1. Using the <code>sqlalchemy</code> library, create a table object for the <code>drug_table</code> table that contains the following fields.<br />
<code>
id (primary key),
medication (string),
route (string),
dose (string),
patient_id (int),
freq per day (int)
</code>
<br />
2. Output the results of the table (print the entire table).<br />
3. Write the following queries:<br />
<ul>
    <li>Return all records containing the drug <code>WARFARIN</code></li>
    <li>Return all records where the frequency per day is more than 0 and less than 3</li>
</ul>
<br />
Don't worry about making <code>patient_id</code> a foreign key for now, just treat it like a regular column.<br />
<br />
<strong>HINT:</strong> for the last task you will need to lookup and use the <code>and&#95;</code> function (<code>from sqlalchemy import and&#95;)</code>. Also for accessing columns that have spaces in their name, you would write <code>drug_table.c['freq per day']</code>.
</div>

In [9]:
drug_table = Table(
   'drug_table', meta, 
   Column('id', Integer, primary_key = True), 
   Column('medication', String), 
   Column('route', String),
   Column('dose', String),
   Column('patient_id', Integer), 
   Column('freq per day', Integer)
)
meta.create_all(engine)

In [10]:
data = drug_table.select()
conn = engine.connect()
result = conn.execute(data)

for each_row in result:
    print(each_row)

(1, 'AMOXICILLIN', 'PO', '500mg', 1, 3)
(2, 'IRBESARTAN', 'PO', '150mg', 2, 1)
(3, 'DIGOXIN', 'PO', '1.5mg', 2, 1)
(4, 'SIMVASTATIN', 'PO', '40', 3, 1)
(5, 'RAMIPRIL', 'PO', '2.5mg', 4, 1)
(6, 'WARFARIN', 'PO', 'variable', 4, 1)
(7, 'SENNA', 'PO', '15mg', 4, 1)
(8, 'None', 'NA', 'NA', 5, 0)


In [12]:
warfarin = drug_table.select().where(drug_table.c.medication == 'WARFARIN')
result = conn.execute(warfarin)

for each_row in result:
    print(each_row)

(6, 'WARFARIN', 'PO', 'variable', 4, 1)


In [27]:
from sqlalchemy import and_

freq = drug_table.select().where(and_(drug_table.c['freq per day'] > 0, drug_table.c['freq per day'] < 3))
result = conn.execute(freq)

for each_row in result:
    print(each_row)

(2, 'IRBESARTAN', 'PO', '150mg', 2, 1)
(3, 'DIGOXIN', 'PO', '1.5mg', 2, 1)
(4, 'SIMVASTATIN', 'PO', '40', 3, 1)
(5, 'RAMIPRIL', 'PO', '2.5mg', 4, 1)
(6, 'WARFARIN', 'PO', 'variable', 4, 1)
(7, 'SENNA', 'PO', '15mg', 4, 1)


In [ ]:
# Type your code here


In [ ]:
# Type your code here


In [ ]:
# Type your code here


In [ ]:
# Type your code here


Another way you might see a table defined is like that below. This is now the SQLAlchemy standard and is referred to as <code>Declarative Mapping</code>.

In [5]:
from sqlalchemy import Table, Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base

Base = declarative_base()

class DrugTable(Base):
    __tablename__ = 'drug_table'
    id = Column(Integer, primary_key = True)
    medication = Column(String)
    route = Column(String)
    dose = Column(String)
    patient_id = Column(Integer)
    freq_per_day = Column(Integer)


Here we are using the standard <code>class</code> to create a table. Each table would have a class. The table name is defined, as are the variables with the columns data types. We can now set the <code>patient_id</code> to be a <code>foreign key</code> to link the two tables.

In [6]:
from sqlalchemy import ForeignKey
from sqlalchemy.orm import relationship

Base = declarative_base()

class MedData(Base):
    __tablename__ = 'med_data'
    id = Column(Integer, primary_key = True)
    name = Column(String)
    age = Column(Integer)
    sex = Column(String)
    blood_pressure = Column("Blood pressure", String)
    heart_rate = Column("Heart rate", String)
    drug_table = relationship("DrugTable", back_populates="med_data")
    
class DrugTable(Base):
    __tablename__ = 'drug_table'
    id = Column(Integer, primary_key = True)
    medication = Column(String)
    route = Column(String)
    dose = Column(String)
    freq_per_day = Column(Integer)
    patient_id = Column(Integer, ForeignKey('med_data.id'))
    med_data = relationship("MedData", back_populates="drug_table")


To create a <code>one-to-many</code> relationship we added the <code>relationship()</code> function to both the <code>MedData</code> and <code>DrugTable</code> classes. The <code>medications</code> variable creates a relationship with the <code>DrugTable</code> and the <code>med_data</code> variable links this with the <code>medications</code> variable. Note that the <code>patient_id</code> is now set as a foreign key using the <code>ForeignKey()</code> function to point to the <code>id</code> column in the <code>MedData</code> class.

To see the different types of relationship patterns, look at this link: <a href="https://docs.sqlalchemy.org/en/13/orm/basic_relationships.html" target="_blank">SQLAlchemy relationships</a>.

<div class="alert alert-success">
<b>Note:</b> Be careful to refer to the class name and table name in the correct places. Mixing these up is a common cause of error and the first thing you might want to check.
</div>

So now we have setup the classes to store the data, lets go ahead and use them. As before we start by creating an engine with the same database file we used previously.

In [7]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///medical_db.db', echo = False)

We start by making an <code>instance</code> of the mapped class.

In [8]:
admission_data = MedData()
medication_data = DrugTable()

We then set up a <code>session</code>. This handles multiple users accessing our database through our application (possibly even at the same time if we have a lot of users).

In [9]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)
session = Session()
session.add(admission_data)

We can now query the database to return all the data.

In [10]:
patients = session.query(MedData).all()
for pt in patients:
    print(f'{pt.name}, {pt.age}, {pt.sex}, {pt.blood_pressure}, {pt.heart_rate}')

Alan Smith, 24, M, 120/70, 78
Maureen Gdiver, 87, F, 156/82, 82
Adam Blythe, 54, M, 132/73, 72
Darren Sanders, 34, M, 155/67, 67
Sally-Ann Joyce, 19, F, 121/72, 65
None, None, None, None, None


We can also carry out specific queries as before. For example lets get all the heart rates more than 70 bpm.

In [11]:
patients = session.query(MedData).filter(MedData.heart_rate > 70).all()
for pt in patients:
    print(f'{pt.name}, {pt.age}, {pt.sex}, {pt.blood_pressure}, {pt.heart_rate}')

Alan Smith, 24, M, 120/70, 78
Maureen Gdiver, 87, F, 156/82, 82
Adam Blythe, 54, M, 132/73, 72


<div class="alert alert-block alert-info">
<b>Task 4:</b>
<br> 
Write a query like the one above to return all the patients aged 30 and under.
</div>

In [57]:
patients = session.query(MedData).filter(MedData.age <= 30).all()
for pt in patients:
    print(f'{pt.name}, {pt.age}, {pt.sex}, {pt.blood_pressure}, {pt.heart_rate}')

Alan Smith, 24, M, 120/70, 78
Sally-Ann Joyce, 19, F, 121/72, 65


In [ ]:
# Type your code here


We can use a join to find all the patients that are taking medications ending with the letters 'IN'.

In [16]:
hypertension_meds = session.query(MedData).join(DrugTable).filter(DrugTable.medication.ilike('%IN')).all()
            
for med in hypertension_meds:
    print(f'{med.name}, {med.age}, {med.sex}')

Alan Smith, 24, M
Maureen Gdiver, 87, F
Adam Blythe, 54, M
Darren Sanders, 34, M


<div class="alert alert-success">
<b>Note:</b> Remember that the results returned are iterable (can be iterated over) and you may need to carry out further processing on the results to get them into the correct format for subsequent use. This is one of the most common issues with using these methods in Python that beginners tend to struggle with. Try outputting (printing) the raw results so you can see what kind of data structure they are returned in. This will give you a clue as to how to process them further to extract the exact required information.
</div>

Finally when we have finished interacting with our database we can close the session.

In [17]:
session.close()

You have come to the end of the SQL notebooks. We have introduced you to the basic SQL commands and taken you right through to using SQL in Python and Object Relational Mapping. When you view the code in the skeleton project we have given you, we hope that this now makes more sense as it was constructed using the techniques we have covered with these notebooks. The dashboard uses ORM to load prescription data from the database for use in the dashboard. The system was built in Python with a web front end to provide a graphical interface to the data in the form of a dashboard. Modern software systems are made using multiple frameworks and libraries to help with the heavy lifting and prevent us having to reinvent the wheel. To use these frameworks and libraries, one must become familiar with their APIs (Application Programing Interfaces). The best way to do this is to start with the documentation. For example here is the documentation for <a href="https://docs.sqlalchemy.org/en/13/orm/tutorial.html#connecting" target="_blank">SQLAlchemy</a>. Sometimes people (including me) often find the documentation less than clear or intuitive. We get round this by searching for simple examples or tutorial blogs/videos on the internet. This is really the best way to learn, and professional software engineers and data scientists do this all the time. We recommend you start to do this if your are not already to help build on your skills and extend your knowledge. 

### Notebook details
<br>
<i>Notebook created by <strong>Dr. Alan Davies</strong>

Publish date: March 2021<br>
Review date: March 2022</i>

Please give your feedback using the button below:

<a class="typeform-share button" href="https://hub11.typeform.com/to/q2cwvhNu" data-mode="popup" style="display:inline-block;text-decoration:none;background-color:#3A7685;color:white;cursor:pointer;font-family:Helvetica,Arial,sans-serif;font-size:18px;line-height:45px;text-align:center;margin:0;height:45px;padding:0px 30px;border-radius:22px;max-width:100%;white-space:nowrap;overflow:hidden;text-overflow:ellipsis;font-weight:bold;-webkit-font-smoothing:antialiased;-moz-osx-font-smoothing:grayscale;" target="_blank">Rate this notebook </a> <script> (function() { var qs,js,q,s,d=document, gi=d.getElementById, ce=d.createElement, gt=d.getElementsByTagName, id="typef_orm_share", b="https://embed.typeform.com/"; if(!gi.call(d,id)){ js=ce.call(d,"script"); js.id=id; js.src=b+"embed.js"; q=gt.call(d,"script")[0]; q.parentNode.insertBefore(js,q) } })() </script>

## Notes: